In [4]:
import gurobipy as grb
import pandas as pd
import numpy as np



In [5]:
!ls

121115.csv  baidu-121115.csv  betting_strategy.ipynb


In [10]:
data = pd.read_csv('121115.csv')
data.columns, data.head()


(Index(['Home vs Away', 'Home Win', 'Home Draw', 'Home Lost', 'b365H', 'b365D',
        'b365A', 'bwinH', 'bwinD', 'bwinA', 'result'],
       dtype='object'),
                     Home vs Away  Home Win  Home Draw  Home Lost  b365H  \
 0        Norwich City vs Everton      0.31       0.32       0.37   3.20   
 1  Crystal Palace vs Southampton      0.33       0.34       0.33   2.40   
 2              West Ham vs Stoke      0.33       0.33       0.34   2.10   
 3            Man City vs Swansea      0.51       0.28       0.21   1.25   
 4          Sunderland vs Watford      0.29       0.31       0.40   2.70   
 
    b365D  b365A  bwinH  bwinD  bwinA  result  
 0   3.30   2.25   3.00   3.30   2.30     NaN  
 1   3.30   2.88   2.50   3.20   2.80     NaN  
 2   3.30   3.50   2.05   3.30   3.60     NaN  
 3   6.00  10.00   1.30   5.25   8.75     NaN  
 4   3.25   2.60   2.70   3.10   2.65     NaN  )

In [75]:
p_games = data[['Home Win', 'Home Draw', 'Home Lost']]
num_of_games = p_games.shape[0]

num_of_results = 3

b365 = data[['b365H', 'b365D', 'b365A']]
bwin = data[['bwinH', 'bwinD', 'bwinA']]
bookmakers = [b365, bwin]
num_of_bookmakers = len(bookmakers)
head_to_head = data['Home vs Away']
names = [['b365'] * num_of_games + ['bwin'] * num_of_games,
          list(head_to_head) * num_of_bookmakers]
bookmakers = pd.DataFrame(np.concatenate((b365.values, bwin.values)), index=names, columns=['Home', 'Draw', 'Away'])
bookmakers.shape

(68, 3)

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [30]:
def kelly_criterion(p, odd):
    return p - (1 - p) / (odd - 1)

In [ ]:
def config_params(total_budget, single_bet_budget, p_games, bookmakers, head_to_head):
    num_of_games = p_games.shape[0]
    num_of_bookmakers = len(bookmakers) / num_of_games
    num_of_results = 3
    p_games_val = p_games.value()
    bookmakers_val = np.zeros((num_of_games, num_of_results, num_of_bookmakers))
    
    for i in np.arange(num_of_bookmakers):
        bookmakers_val[:, :, i] = bookmakers[i].value()
    
    # Model
    m = grb.Model('betting strategy')
    
    # Kelly, bookmakers, and probabilities of the results of each game
    kellies = np.zeros((1, num_of_games * num_of_bookmakers * num_of_results))
    bookmaker_row_vector = np.zeros((1, num_of_games * num_of_bookmakers * num_of_results))
    prob_row_vector = np.zeros((1, num_of_games * num_of_bookmakers * num_of_results))
    
    # Create decision variables for the betting strategy, which are binary vars
    # either bet this one or not. The price to bet is the corresponding kelly criteria value
    # Also populate the parameters here
    var = {}
    var_idx = 0
    n = 0
    for i in np.arange(num_of_games):
        for j in np.arange(num_of_bookmakers):
            for k in np.arange(num_of_results):
                p = p_games_val[i, k]
                odd = bookmakers_val[i, k, j]
                kelly = kelly_criterion(p, odd)
                kellies[0, var_idx] = kelly
                bookmaker_row_vector[0, var_idx] = odd
                prob_row_vector[0, var_idx] = p
                var[head_to_head[i] + ',' + p_games.columns[k] + ',' + ]
                
                var_idx += 1
                